In [1]:
!pip install qiskit qiskit-aer --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00


In [2]:
# Task 1: Control/Target Variation

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

# Create a 2-qubit circuit
qc1 = QuantumCircuit(2, 2)

# --- Apply multi-qubit gates with swapped control/target ---
qc1.h(1)        # Hadamard on qubit 1 instead of qubit 0
qc1.cx(1, 0)    # CNOT with control qubit 1 and target qubit 0
qc1.cz(1, 0)    # Controlled-Z with control qubit 1 and target qubit 0
qc1.swap(0, 1)  # Swap qubits 0 and 1

# Measure all qubits
qc1.measure_all()

# Display the circuit
print("Quantum Circuit (Control/Target Swapped):")
print(qc1.draw())

# Run simulation
compiled1 = transpile(qc1, sim)
result1 = sim.run(compiled1, shots=1024).result()

# Display results
counts1 = result1.get_counts()
print("\nMeasurement Counts:", counts1)


Quantum Circuit (Control/Target Swapped):
             ┌───┐       ░ ┌─┐   
   q_0: ─────┤ X ├─■──X──░─┤M├───
        ┌───┐└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ┤ H ├──■───■──X──░──╫─┤M├
        └───┘            ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 

Measurement Counts: {'11 00': 511, '00 00': 513}


In [3]:
# Task 2: Gate Removal (Remove SWAP gate)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

# Create a 2-qubit circuit without SWAP gate
qc2 = QuantumCircuit(2, 2)

qc2.h(0)        # Hadamard on qubit 0
qc2.cx(0, 1)    # CNOT with control 0 and target 1
qc2.cz(0, 1)    # Controlled-Z
# qc2.swap(0, 1)  # SWAP gate removed

# Measure all qubits
qc2.measure_all()

# Display the circuit
print("Quantum Circuit (SWAP gate removed):")
print(qc2.draw())

# Run simulation
compiled2 = transpile(qc2, sim)
result2 = sim.run(compiled2, shots=1024).result()

# Display results
counts2 = result2.get_counts()
print("\nMeasurement Counts:", counts2)

# Explanation:
# The SWAP gate exchanges the states of two qubits.
# Removing it changes which qubit ends up in which state at measurement.
# Without SWAP, qubit 0 remains as control and qubit 1 as target throughout the circuit,
# so the output bit order and correlation will differ from the original.


Quantum Circuit (SWAP gate removed):
        ┌───┐         ░ ┌─┐   
   q_0: ┤ H ├──■───■──░─┤M├───
        └───┘┌─┴─┐ │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──░──╫─┤M├
             └───┘    ░  ║ └╥┘
   c: 2/═════════════════╬══╬═
                         ║  ║ 
meas: 2/═════════════════╩══╩═
                         0  1 

Measurement Counts: {'11 00': 514, '00 00': 510}


In [4]:
# Task 3: Add Another Hadamard (on second qubit before CNOT)

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize simulator
sim = AerSimulator()

# Create a 2-qubit circuit
qc3 = QuantumCircuit(2, 2)

qc3.h(0)        # Hadamard on qubit 0
qc3.h(1)        # Additional Hadamard on qubit 1 (new)
qc3.cx(0, 1)    # CNOT with control 0, target 1
qc3.cz(0, 1)    # Controlled-Z
qc3.swap(0, 1)  # SWAP qubits

# Measure all qubits
qc3.measure_all()

# Display the circuit
print("Quantum Circuit (Additional H on Qubit 1):")
print(qc3.draw())

# Run simulation
compiled3 = transpile(qc3, sim)
result3 = sim.run(compiled3, shots=1024).result()

# Display results
counts3 = result3.get_counts()
print("\nMeasurement Counts:", counts3)

# Explanation:
# Adding another Hadamard on qubit 1 introduces superposition before entanglement.
# This increases the number of possible interference patterns between qubits.
# The result is often a more "mixed" or balanced output distribution across all states (00, 01, 10, 11),
# compared to the original circuit which had more predictable correlations.


Quantum Circuit (Additional H on Qubit 1):
        ┌───┐            ░ ┌─┐   
   q_0: ┤ H ├──■───■──X──░─┤M├───
        ├───┤┌─┴─┐ │  │  ░ └╥┘┌─┐
   q_1: ┤ H ├┤ X ├─■──X──░──╫─┤M├
        └───┘└───┘       ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 

Measurement Counts: {'11 00': 260, '10 00': 238, '00 00': 266, '01 00': 260}
